In [1]:
import pickle
import re
import spacy
from spacy import displacy
import pandas as pd
nlp = spacy.load("en_core_web_sm")

# Import Reviews as DataFrame

In [2]:
data = pd.read_csv('reviews.csv',index_col=[0])
data = data.dropna()

#data = "Affordable Korean Food At Funan. Beef kalbi was disappointing, kimchi pancake is on the saltier side.... Fried chicken is normal..."

# Init functions to remove and extract hashtags

In [3]:
def removeHashTags(review_str):
    
    '''
    Input: Review in string format.
    Output: String of the review with starting hashtags all removed.
            (hashtags in the middle of reviews will not be removed
            to preserve phrasing)
    
    E.g.
    
    removeHashTags("#Amazing #Delicious #Foodcoma #iHateHashTags The wagyu beef was the #BEST i've every tried")
    
    >> The wagyu beef was the BEST i've ever tried.
    '''
    
    review = review_str.split()
    
    temp = 0
    
    for i in range(len(review)):
        
        text = review[i]
        
        if text[0] == '#' and temp == i:
            review[i] = ' '
            temp += 1
            
        elif text[0] == '#':
            review[i] = review[i].lstrip('#')
    
    return ' '.join(review).lstrip()


###-------------------------------------------###
###-------------------------------------------###
###-------------------------------------------###


def getHashTags(review_str):
    
    '''
    Input: review text in string format.
    Output: List of HashTags
    
    e.g. getHashTags('The #amazing wagyu #beef was so #juicy')
    >>> [#amazing, #beef, #juicy]
    '''
    
    return re.findall('#\w+',review_str)

In [5]:
data

,ReviewTitle,RestaurantAddress,RestaurantName,Review
0,"""Life is a combination of magic and noodles.""","\n11 Lorong 3 Toa Payoh, Singapore\n",\nChef Kang’s Noodle House\n,Featuring Wonton Noodles from Chef Kang's Nood...
1,#SoNotHungry 🤷🏻‍♀️ .,"\n215R Upper Thomson Road, Singapore\n",\nOne Man Coffee (Upper Thomson)\n,@igsg #igsg #singapore #foodpornasia @burpple ...
2,#Throwback to dinner at the new @mimirestauran...,"\n3A River Valley Road, Singapore\n",\nMimi\n,"Interesting ambience, great service, and delec..."
8,#coldbrewsaturdays for hot weather with @boots...,\nSingapore\n,\nSingapore\n,The coffee is acceptably strong as well!
12,"#jazpsterseoul 🇰🇷\nAbsolutely divine, robust #...",\nChangcheon-dong\n,\n파이홀\n,"Cozy space, so expect to wait for seats during..."
...,...,...,...,...
1076,😋 (Review2),"\n 26 Beach Road, Singapore\n",\nWild Honey (South Beach)\n,My 3rd visit to Wild Honey!\nThis time we inte...
1077,😘Friends that eat together stay together!,"\n3 Woodlands Street 13, Singapore\n",\nCitrus By The Pool\n,@citrusbythepool is a Halal Certified cafe tha...
1078,🙄,"\n3 Temasek Boulevard, Singapore\n",\nThe Salted Plum (Suntec City)\n,Finally tried this ‘famous’ Taiwanese stall th...
1079,🥰Check out our delicious Dinner- Unagi Kabayak...,"\n5 Burn Road, Singapore\n",\nGrain\n,Have you tried these new dishes yet?🥰\n.\nGrai...


# Get Nouns

A 2-stage process is used to extract all nouns and adjective phrases.

<ol>
    <li>We __extract all unigram nouns__ and append it into the "results" list.</li>
<li>Next, we __extract bigram and trigram units__ that have predefined POS tag sequences, and continue appending the extracted results to the "results" list. The current implementation below uses 7 rules.</li>
 </ol>

In [6]:
# Store all results in this list
results = []

for i in range(data.shape[0]):
    rev = data['Review'].iloc[i]
    rev = rev.replace('#','')
    doc = nlp(rev)

    for w in doc:
        if w.pos_ == 'NOUN':
            results.append(str(w))

# POS and Dependency Extraction

In [7]:
# For each review
for n in range(data.shape[0]):
    
    rev = data['Review'].iloc[n]
        
    try:
        rev = rev.replace('\n', ' ')
    except:
        continue

    rev = removeHashTags(rev)
    doc = nlp(rev)

    text_list = []
    pos_list =[]
    dep_list=[]

    # For each word in the review
    for w in doc:

        extract = None

        text_list.append(str(w.text))
        pos_list.append(w.pos_)
        dep_list.append(w.dep_)

        if len(text_list) > 3:
            # Rule 1
            if pos_list[-1] in ['NOUN', 'PROPN'] and pos_list[-2] in ['NOUN', 'PROPN'] and pos_list[-3] == 'ADJ':
                extract = text_list[-3] + ' ' +  text_list[-2] + ' ' + text_list[-1]
                print('Rule 1: ',extract,'({} {} {})'.format(pos_list[-3], pos_list[-2], pos_list[-1]))

            # Rule 2
            elif pos_list[-1] in ['NOUN', 'PROPN'] and pos_list[-2] in ['NOUN', 'PROPN'] and pos_list[-3] == 'VERB':
                extract = text_list[-3] + ' ' + text_list[-2] + ' ' + text_list[-1]
                print('Rule 2 ',extract,'({} {} {})'.format(pos_list[-3], pos_list[-2], pos_list[-1]))

            # Rule 3
            elif pos_list[-1] in ['NOUN', 'PROPN'] and pos_list[-2] in ['NOUN', 'PROPN'] and pos_list[-3] == 'NOUN':
                extract = text_list[-3] + ' ' + text_list[-2] + ' ' + text_list[-1]
                print('Rule 3: ',extract,'({} {} {})'.format(pos_list[-3], pos_list[-2], pos_list[-1]))

            # Rule 4
            elif pos_list[-1] in ['NOUN', 'PROPN'] and pos_list[-2] in ['NOUN', 'PROPN']:
                extract = text_list[-2] + ' ' + text_list[-1]
                print('Rule 4: ',extract,'({} {})'.format(pos_list[-2], pos_list[-1]))

            # Rule 5
            elif pos_list[-1] in ['NOUN', 'PROPN'] and pos_list[-2] == 'VERB':
                extract = text_list[-2] + ' ' + text_list[-1]
                print('Rule 5: ',extract,'({} {})'.format(pos_list[-2], pos_list[-1]))

            # Rule 6
            elif pos_list[-1] in ['NOUN', 'PROPN'] and pos_list[-2] == 'ADJ':
                extract = text_list[-2] + ' ' + text_list[-1]
                print('Rule 6: ',extract,'({} {})'.format(pos_list[-2], pos_list[-1]))

            # Rule 7
            if pos_list[-1] in ['NOUN', 'PROPN'] and dep_list[-1]=='dobj':
                extract = text_list[-2] + ' ' + text_list[-1]
                print('Rule 7: ',extract,'({} {})'.format(pos_list[-2], pos_list[-1]))

            if extract != None:

                results.append(extract)


Rule 4:  Chef Kang (PROPN PROPN)
Rule 4:  Noodle House (PROPN PROPN)
Rule 3:  igsg singapore foodpornasia (NOUN NOUN NOUN)
Rule 6:  @burpple burpple (ADJ NOUN)
Rule 1:  @burpple burpple burpplebeyond (ADJ NOUN NOUN)
Rule 6:  @chopesg chopesg (ADJ NOUN)
Rule 1:  @chopesg chopesg chopebites (ADJ NOUN NOUN)
Rule 3:  chopesg chopebites chopedeals (NOUN NOUN NOUN)
Rule 7:  chopebites chopedeals (NOUN NOUN)
Rule 4:  hungrygowhere setheats (NOUN NOUN)
Rule 3:  hungrygowhere setheats eatoutsg (NOUN NOUN NOUN)
Rule 6:  sgfoodie @sgfoodie (ADJ NOUN)
Rule 1:  sgfoodie @sgfoodie @singaporeeats (ADJ NOUN NOUN)
Rule 3:  @sgfoodie @singaporeeats singaporeeats (NOUN NOUN NOUN)
Rule 3:  @singaporeeats singaporeeats sgcafe (NOUN NOUN NOUN)
Rule 5:  sgcafehopping sgcafefood (VERB NOUN)
Rule 2  sgcafehopping sgcafefood @cafehoppingsg (VERB NOUN PROPN)
Rule 3:  sgcafefood @cafehoppingsg breakfast (NOUN PROPN NOUN)
Rule 4:  breakfast brunch (NOUN NOUN)
Rule 3:  breakfast brunch bigbreakfast (NOUN NOUN NOUN)

Rule 4:  ▪ Egg (PROPN PROPN)
Rule 4:  Egg Toast (PROPN PROPN)
Rule 4:  Toast Beef (PROPN PROPN)
Rule 4:  Beef Bulgogi (PROPN PROPN)
Rule 4:  Ala Carte (PROPN PROPN)
Rule 4:  ▪ Egg (PROPN PROPN)
Rule 4:  Egg Toast (PROPN PROPN)
Rule 4:  Toast Ham (PROPN PROPN)
Rule 4:  Ala Carte (PROPN PROPN)
Rule 4:  ▪ Egg (PROPN PROPN)
Rule 4:  Egg Toast (PROPN PROPN)
Rule 4:  Toast Bacon (PROPN PROPN)
Rule 4:  Ala Carte (PROPN PROPN)
Rule 4:  @burgerplussg Breakfast (PROPN PROPN)
Rule 4:  Breakfast menu (PROPN NOUN)
Rule 4:  @burgerplusg Wisma (PROPN PROPN)
Rule 4:  Wisma Atria (PROPN PROPN)
Rule 4:  Orchard Road (PROPN PROPN)
Rule 4:  burgerplussg burpple (NOUN NOUN)
Rule 3:  burgerplussg burpple burpplesg (NOUN NOUN NOUN)
Rule 4:  Tteokgangjeong Combo (PROPN PROPN)
Rule 4:  Combo Set (PROPN PROPN)
Rule 4:  Cheesetos Chicken (PROPN PROPN)
Rule 4:  Chicken Combo (PROPN PROPN)
Rule 4:  Combo Set (PROPN PROPN)
Rule 7:  burgerplus burgerplussg (CCONJ NOUN)
Rule 4:  burgerplussg wismaatria (NOUN NOUN)
Ru

Rule 2  foodstyling insiderfood#f52grams burpple (VERB NOUN NOUN)
Rule 5:  exploreflavours dessert (VERB NOUN)
Rule 6:  sweettooth buzzfeed (ADJ NOUN)
Rule 6:  singaporefood canonsg (ADJ NOUN)
Rule 4:  Intestine Mee (PROPN PROPN)
Rule 4:  Mee Sua (PROPN PROPN)
Rule 7:  3 Bowls (NUM NOUN)
Rule 6:  good alternative (ADJ NOUN)
Rule 6:  fancy oyster (ADJ NOUN)
Rule 1:  fancy oyster mee (ADJ NOUN NOUN)
Rule 7:  oyster mee (NOUN NOUN)
Rule 3:  oyster mee sua (NOUN NOUN NOUN)
Rule 6:  good eat (ADJ NOUN)
Rule 6:  mid joints (ADJ NOUN)
Rule 7:  mid joints (ADJ NOUN)
Rule 6:  flavourful touch (ADJ NOUN)
Rule 6:  tumeric spices (ADJ NOUN)
Rule 6:  clever twist (ADJ NOUN)
Rule 6:  classic staple (ADJ NOUN)
Rule 6:  nasi lemak (ADJ NOUN)
Rule 6:  comforting congee (ADJ NOUN)
Rule 1:  comforting congee dip (ADJ NOUN NOUN)
Rule 6:  nothingisordinary myfab5 (ADJ NOUN)
Rule 1:  nothingisordinary myfab5 f52grams (ADJ NOUN NOUN)
Rule 6:  lovefood eatingfortheinsta (ADJ NOUN)
Rule 1:  lovefood eatingfort

Rule 7:  accepts GrabPay (VERB PROPN)
Rule 6:  cashless transactions (ADJ NOUN)
Rule 5:  earning Grab (VERB PROPN)
Rule 2  earning Grab points (VERB PROPN NOUN)
Rule 7:  Grab points (PROPN NOUN)
Rule 6:  my way (ADJ NOUN)
Rule 7:  my way (ADJ NOUN)
Rule 6:  frozen goodies (ADJ NOUN)
Rule 4:  Xiao Long (PROPN PROPN)
Rule 4:  Long Baos (PROPN PROPN)
Rule 4:  Baos 小笼包 (PROPN NOUN)
Rule 4:  dinner menu (NOUN NOUN)
Rule 6:  My eyes (ADJ NOUN)
Rule 5:  saw Avocado (VERB PROPN)
Rule 7:  saw Avocado (VERB PROPN)
Rule 6:  wonderful breakfast (ADJ NOUN)
Rule 1:  wonderful breakfast option (ADJ NOUN NOUN)
Rule 4:  avocado lover (NOUN NOUN)
Rule 6:  Creamy slices (ADJ NOUN)
Rule 5:  grilled halloumi (VERB NOUN)
Rule 6:  multigrain sourdough (ADJ NOUN)
Rule 6:  baked pumpkin (ADJ NOUN)
Rule 6:  micro greens (ADJ NOUN)
Rule 6:  great post (ADJ NOUN)
Rule 4:  workout item (NOUN NOUN)
Rule 4:  avocado 🥑 (NOUN PROPN)
Rule 3:  avocado 🥑 chicken (NOUN PROPN NOUN)
Rule 4:  chicken slices (NOUN NOUN)
Rule 

Rule 5:  foodstamping sgfood (VERB NOUN)
Rule 2  foodstamping sgfood foodoftheday (VERB NOUN NOUN)
Rule 3:  sgfood foodoftheday foodporn (NOUN NOUN NOUN)
Rule 3:  foodoftheday foodporn burpple (NOUN NOUN NOUN)
Rule 5:  foodspotting stfood (VERB NOUN)
Rule 5:  igfood instafood (VERB NOUN)
Rule 2  igfood instafood openricesg (VERB NOUN NOUN)
Rule 3:  instafood openricesg foodphotography (NOUN NOUN NOUN)
Rule 7:  openricesg foodphotography (NOUN NOUN)
Rule 6:  umakemehungry TSLMakan (ADJ PROPN)
Rule 5:  foodblogs nomnomnom (VERB NOUN)
Rule 6:  yelpsg boatnoodle (ADJ NOUN)
Rule 1:  yelpsg boatnoodle bugisjunction (ADJ NOUN NOUN)
Rule 5:  halal boatboodlesg (VERB NOUN)
Rule 2  halal boatboodlesg capitamall (VERB NOUN NOUN)
Rule 7:  boatboodlesg capitamall (NOUN NOUN)
Rule 4:  boba emoji (NOUN NOUN)
Rule 4:  🤔 🤔 (PROPN PROPN)
Rule 4:  🤔 🤔 (PROPN PROPN)
Rule 6:  thealley bubbletea (ADJ NOUN)
Rule 1:  thealley bubbletea boba (ADJ NOUN NOUN)
Rule 5:  foodinhands foodphotography (VERB NOUN)
Rule

...

Rule 7:  good find (ADJ NOUN)
Rule 7:  a table (DET NOUN)
Rule 6:  next time (ADJ NOUN)
Rule 4:  money breakfast (NOUN NOUN)
Rule 4:  freak cake (NOUN NOUN)
Rule 3:  freak cake 🍵 (NOUN NOUN NOUN)
Rule 5:  styled breakfast (VERB NOUN)
Rule 4:  Kale Haloumi (PROPN PROPN)
Rule 4:  Haloumi Quinoa (PROPN PROPN)
Rule 6:  main ingredients (ADJ NOUN)
Rule 7:  main ingredients (ADJ NOUN)
Rule 6:  Other ingredients (ADJ NOUN)
Rule 7:  all things (DET NOUN)
Rule 6:  poached egg (ADJ NOUN)
Rule 6:  multigrain bread (ADJ NOUN)
Rule 6:  sinful food (ADJ NOUN)
Rule 1:  sinful food 😝 (ADJ NOUN NOUN)
Rule 3:  food 😝 😝 (NOUN NOUN PROPN)
Rule 4:  🤣 👌 (NOUN NOUN)
Rule 6:  singaporefood food (ADJ NOUN)
Rule 6:  yum instafood (ADJ NOUN)
Rule 1:  yum instafood TagsForLikes (ADJ NOUN NOUN)
Rule 6:  instagood photooftheday (ADJ NOUN)
Rule 6:  sweet dinner (ADJ NOUN)
Rule 1:  sweet dinner lunch (ADJ NOUN NOUN)
Rule 3:  dinner lunch breakfast (NOUN NOUN NOUN)
Rule 5:  eating foodpic (VERB NOUN)
Rule 2  eating fo

Rule 3:  sgig cafehoppingsg thedailybite (NOUN NOUN NOUN)
Rule 3:  cafehoppingsg thedailybite makan (NOUN NOUN NOUN)
Rule 5:  stfoodtrending wantonmee (VERB NOUN)
Rule 6:  uncagestreetfood sghawker (ADJ NOUN)
Rule 1:  uncagestreetfood sghawker hawkerfood (ADJ NOUN NOUN)
Rule 6:  singaporefood cbdeats (ADJ NOUN)
Rule 7:  singaporefood cbdeats (ADJ NOUN)
Rule 6:  French influences (ADJ NOUN)
Rule 5:  cut fillets (VERB NOUN)
Rule 7:  cut fillets (VERB NOUN)
Rule 6:  bonito fish (ADJ NOUN)
Rule 4:  rice straws (NOUN NOUN)
Rule 6:  sliced cucumber (ADJ NOUN)
Rule 6:  flaky meaty (ADJ NOUN)
Rule 1:  flaky meaty chew (ADJ NOUN NOUN)
Rule 7:  meaty chew (NOUN NOUN)
Rule 6:  bonito fish (ADJ NOUN)
Rule 6:  deep savoury (ADJ NOUN)
Rule 1:  deep savoury salty  (ADJ PROPN)
Rule 6:  👍 🏻 (ADJ PROPN)
Rule 4:  Chinatown Complex (PROPN PROPN)
Rule 4:  Complex Market (PROPN PROPN)
Rule 4:  Food Centre (PROPN PROPN)
Rule 4:  Smith Street (PROPN PROPN)
Rule 4:  Asian Kong (PROPN PROPN)
Rule 4:  Kong Bak (

Rule 4:  Shenton Way (PROPN PROPN)
Rule 4:  Nearest MRT (PROPN PROPN)
Rule 4:  Tanjong Pagar (PROPN PROPN)
Rule 4:  EW Line (PROPN NOUN)
Rule 4:  Opening Hours (NOUN PROPN)
Rule 4:  pm Sat (NOUN PROPN)
Rule 6:  beautiful coat (ADJ NOUN)
Rule 7:  beautiful coat (ADJ NOUN)
Rule 6:  sticky sweetness (ADJ NOUN)
Rule 4:  folk tender (NOUN NOUN)
Rule 3:  folk tender pork (NOUN NOUN NOUN)
Rule 3:  tender pork ribs (NOUN NOUN NOUN)
Rule 7:  pork ribs (NOUN NOUN)
Rule 4:  steam rice (NOUN NOUN)
Rule 4:  Luck Restaurant (PROPN PROPN)
Rule 4:  Holland Ave (PROPN PROPN)
Rule 4:  Ave Singapore (PROPN PROPN)
Rule 4:  Nearest MRT (PROPN PROPN)
Rule 4:  Holland Village (PROPN PROPN)
Rule 4:  CC Line (PROPN NOUN)
Rule 4:  Opening Hours (NOUN NOUN)
Rule 5:  do breakfasts (VERB NOUN)
Rule 7:  do breakfasts (VERB NOUN)
Rule 4:  Siloso Beach (PROPN PROPN)
Rule 6:  • GIVEAWAY (ADJ PROPN)
Rule 7:  a chance (DET NOUN)
Rule 4:  Siloso Beach (PROPN PROPN)
Rule 5:  Follow @trapizza.sg (VERB NOUN)
Rule 7:  Follow

Rule 5:  dessertbyquintessa dessert (VERB NOUN)
Rule 2  dessertbyquintessa dessert nutella (VERB NOUN NOUN)
Rule 3:  dessert nutella redvelvet (NOUN NOUN NOUN)
Rule 6:  caramel ondeh (ADJ NOUN)
Rule 1:  caramel ondeh cupcakes (ADJ NOUN NOUN)
Rule 3:  ondeh cupcakes bakesale (NOUN NOUN NOUN)
Rule 7:  cupcakes bakesale (NOUN NOUN)
Rule 4:  IG post (PROPN NOUN)
Rule 6:  best baker (ADJ NOUN)
Rule 6:  very fact (ADJ NOUN)
Rule 6:  my cupcakes (ADJ NOUN)
Rule 5:  make someone (VERB NOUN)
Rule 7:  's day (PART NOUN)
Rule 5:  is plenty (VERB NOUN)
Rule 4:  Thanks Liz (NOUN PROPN)
Rule 6:  your colleagues (ADJ NOUN)
Rule 6:  new Ondeh (ADJ PROPN)
Rule 4:  Ondeh Cupcakes (PROPN PROPN)
Rule 4:  Cupcakes 😘 (PROPN NOUN)
Rule 4:  😘 foodphotography (NOUN NOUN)
Rule 3:  😘 foodphotography foodporn (NOUN NOUN NOUN)
Rule 3:  foodphotography foodporn foodgasm (NOUN NOUN NOUN)
Rule 5:  dessertbyquintessa dessert (VERB NOUN)
Rule 7:  dessertbyquintessa dessert (VERB NOUN)
Rule 2  dessertbyquintessa dessert

Rule 3:  instafoodsg igfoodie iweeklyfood (NOUN NOUN NOUN)
Rule 3:  igfoodie iweeklyfood foodcoma (NOUN NOUN NOUN)
Rule 7:  iweeklyfood foodcoma (NOUN NOUN)
Rule 5:  foodstamping sgblogger (VERB NOUN)
Rule 6:  iphone6s vscocam (ADJ NOUN)
Rule 6:  burpplesg openrice (ADJ NOUN)
Rule 1:  burpplesg openrice openricesg (ADJ NOUN NOUN)
Rule 3:  openrice openricesg whati8today (NOUN NOUN PROPN)
Rule 3:  openricesg whati8today hungrygowhere (NOUN PROPN NOUN)
Rule 4:  hungrygowhere cafesg (NOUN NOUN)
Rule 3:  hungrygowhere cafesg cafehoppingsg (NOUN NOUN NOUN)
Rule 3:  cafesg cafehoppingsg sgfoodblogger (NOUN NOUN NOUN)
Rule 6:  foodie foodies (ADJ NOUN)
Rule 7:  foodie foodies (ADJ NOUN)
Rule 5:  sgfoodporn sgfoodunion (VERB NOUN)
Rule 2  sgfoodporn sgfoodunion instafoodsg (VERB NOUN NOUN)
Rule 3:  sgfoodunion instafoodsg igfoodie (NOUN NOUN NOUN)
Rule 3:  instafoodsg igfoodie iweeklyfood (NOUN NOUN NOUN)
Rule 3:  igfoodie iweeklyfood foodcoma (NOUN NOUN NOUN)
Rule 7:  iweeklyfood foodcoma (NO

# Format and export results as txt file

Returns all unique elements in the "results" list, and sorts all elements from A-Z

In [8]:
unique_results = sorted(list(set(results)))

# Export
with open('master_wordlist3.txt', 'w', encoding='utf-8') as f:
    for text in unique_results:
        f.write(text +'\n')

print("Export successful: {} phrases".format(len(unique_results)))

Export successful: 12137 phrases


In [26]:
# count words

print('Original words count: ', len(re.findall(r'\w+', ''.join(data['Review']))))
print('Processed words count:', len(re.findall(r'\w+', ''.join(unique_results))))

with open('food_list_V2.txt',encoding="utf8") as f:
    food_list = [x.strip() for x in f.readlines()]

print('Cleaned word count:', len(re.findall(r'\w+', ''.join(food_list))))

Original words count:  50189
Processed words count: 10674
Cleaned word count: 611
